In [2]:
import tabula
from unicodedata import category

pdf_file = "PDF/d_pos_v6.pdf"
#pdf_file = "PDF/d_neg_v6.pdf"
dfs = tabula.read_pdf(pdf_file, pages="all", stream=True, encoding='cp1252')
tabula.environment_info()
print(len(dfs))

Failed to import jpype dependencies. Fallback to subprocess.
No module named 'jpype'


Python version:
    3.13.3 (tags/v3.13.3:6280bb5, Apr  8 2025, 14:47:33) [MSC v.1943 64 bit (AMD64)]
Java version:
    openjdk version "21.0.1" 2023-10-17 LTS
OpenJDK Runtime Environment Microsoft-8526870 (build 21.0.1+12-LTS)
OpenJDK 64-Bit Server VM Microsoft-8526870 (build 21.0.1+12-LTS, mixed mode, sharing)
tabula-py version: 2.10.0
platform: Windows-11-10.0.26100-SP0
uname:
    uname_result(system='Windows', node='DESKTOP-4A7A8NJ', release='11', version='10.0.26100', machine='AMD64')
linux_distribution: ('', '', '')
mac_ver: ('', ('', '', ''), '')
3


In [3]:
print(len(dfs))

3


In [4]:
dfs[len(dfs)-1]

,Gene,Zygosity,Inheritance,HGVS Description,Location: GRCh38 (hg38),*Classification
0,BRCA1,Heterozygous,Autosomal,c.3143delG p.(Gly1048ValfsTer14),17: 43092388 (GRCh38),Pathogenic
1,NaN,NaN,dominant,NaN,NaN,NaN
2,BRCA2,No clinically,NaN,NaN,NaN,Not
3,NaN,pathogenic,NaN,NaN,NaN,applicable
4,NaN,variant detected,NaN,NaN,NaN,NaN
5,PALB2,No clinically,NaN,NaN,NaN,Not
6,NaN,pathogenic,NaN,NaN,NaN,applicable
7,NaN,variant detected,NaN,NaN,NaN,NaN


Based on https://build.fhir.org/ig/HL7/genomics-reporting/

I think we can build a variant http://hl7.org/fhir/uv/genomics-reporting/StructureDefinition/variant

In [44]:
from fhir.resources.observation import Observation,ObservationComponent
from fhir.resources.codeableconcept import CodeableConcept
from fhir.resources.coding import Coding

df = dfs[len(dfs)-1]
df['Gene'] = df['Gene'].astype(str)
df.dtypes

Gene                       object
Zygosity                   object
Inheritance                object
HGVS Description           object
Location: GRCh38 (hg38)    object
*Classification            object
dtype: object

In [50]:


for i in range(0, len(df)):
    if df.loc[i,'Gene'] is not None and df.loc[i,'Gene'] != 'nan':
        print('\n',df.loc[i,'Gene'],'\n')
        observationBase = {
            "status" : "final",
            "category":[
                {
                    "coding" : [
                        {
                            "system" : "http://terminology.hl7.org/CodeSystem/observation-category",
                            "code" : "laboratory"
                        }
                    ]
                },
                {
                    "coding" : [
                        {
                            "system" : "http://terminology.hl7.org/CodeSystem/v2-0074",
                            "code" : "GE"
                        }
                    ]
                }
            ],
            "code" : {
                "coding" : [
                    {
                        "system" : "http://loinc.org",
                        "code" : "69548-6",
                        "display" : "Genetic variant assessment"
                    }
                ]
            },
            "valueCodeableConcept" : {
                "coding" : [
                    {
                        "system" : "http://loinc.org",
                        "code" : "LA9633-4",
                        "display" : "Present"
                    }
                ]
            },
            "method" : {
                "coding" : [
                    {
                        "system" : "http://loinc.org",
                        "code" : "LA26398-0",
                        "display" : "Sequencing"
                    }
                ]
            },
            "component" : []
        }
        obs = Observation(**observationBase)
        genename = "NK"
        if df.loc[i,'Gene'] == 'BRCA1':
            genename = "HGNC:1100"
        if df.loc[i,'Gene'] == 'BRCA2':
            genename = "HGNC:1101"
        if df.loc[i,'Gene'] == 'PALB2':
            genename = "HGNC:26144"
        geneData = {
            "code" : {
                "coding" : [
                    {
                        "system" : "http://loinc.org",
                        "code" : "48018-6"
                    }
                ]
            },
            "valueCodeableConcept" : {
                "coding" : [
                    {
                        "system" : "http://www.genenames.org",
                        "code" : genename,
                        "display" : df.loc[i,'Gene']
                    }
                ]
            }
        }
        gene = ObservationComponent(**geneData)
        obs.component.append(gene)

        if df.loc[i,'Zygosity'] == 'Heterozygous':
            alleData = {
                "code" : {
                    "coding" : [
                        {
                            "system" : "http://loinc.org",
                            "code" : "53034-5"
                        }
                    ]
                },
                "valueCodeableConcept" : {
                    "coding" : [
                        {
                            "system" : "http://loinc.org",
                            "code" : "LA6706-1",
                            "display" : "Heterozygous"
                        }
                    ]
                }
            }
            alle = ObservationComponent(**alleData)
            obs.component.append(alle)



        json_str = obs.model_dump_json()
        print(json_str)


 BRCA1 

{"resourceType":"Observation","status":"final","category":[{"coding":[{"system":"http://terminology.hl7.org/CodeSystem/observation-category","code":"laboratory"}]},{"coding":[{"system":"http://terminology.hl7.org/CodeSystem/v2-0074","code":"GE"}]}],"code":{"coding":[{"system":"http://loinc.org","code":"69548-6","display":"Genetic variant assessment"}]},"valueCodeableConcept":{"coding":[{"system":"http://loinc.org","code":"LA9633-4","display":"Present"}]},"method":{"coding":[{"system":"http://loinc.org","code":"LA26398-0","display":"Sequencing"}]},"component":[{"code":{"coding":[{"system":"http://loinc.org","code":"48018-6"}]},"valueCodeableConcept":{"coding":[{"system":"http://www.genenames.org","code":"HGNC:1100","display":"BRCA1"}]}},{"code":{"coding":[{"system":"http://loinc.org","code":"53034-5"}]},"valueCodeableConcept":{"coding":[{"system":"http://loinc.org","code":"LA6706-1","display":"Heterozygous"}]}}]}

 BRCA2 

{"resourceType":"Observation","status":"final","catego